<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231124-%E5%88%92%E5%88%86%E6%B5%8B%E8%AF%95%E5%92%8C%E9%AA%8C%E8%AF%81%E6%89%93%E5%8D%B0%E4%B8%80%E4%B8%AA%E7%82%B9df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

# 筛选Sentinel-2影像

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-05-01').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

873


# HANTS

In [5]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

In [6]:
# 一阶谐波
harmonicIndependents_1 = ee.List(['constant', 't', 'cos1', 'sin1'])

def add_sincos_1(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1'))

harmonicSentinel_1 = dataset.map(add_sincos_1)

harmonicTrend_1 = harmonicSentinel_1.select(harmonicIndependents_1.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_1.length(),numY=1))

harmonicTrendCoefficients_1 = harmonicTrend_1.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_1])

def fit_1(image):
  return image.addBands(image.select(harmonicIndependents_1).multiply(harmonicTrendCoefficients_1).reduce('sum').rename('fitted'))

fittedHarmonic_1 = harmonicSentinel_1.map(fit_1)

In [30]:
composite1 = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-24').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)
filteredSentinel1 = composite1.map(s2addVariables)
dataset1 = filteredSentinel1.select(['NDVI','t','constant']);
harmonicSentinel1 = dataset1.map(add_sincos_1)
fittedHarmonic1 = harmonicSentinel1.map(fit_1)

In [31]:
point = ee.Geometry.Point(103.83599109837463,30.889903252109654)

In [32]:
rTable = ee.FeatureCollection(fittedHarmonic1.filterBounds(point).map(lambda image: ee.Feature(
    None,
    {
        'date': image.date().format('yyyy-MM-dd'),
        'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
        'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
    }
)))

rTable_dict = rTable.getInfo()

dates = [feat['properties']['date'] for feat in rTable_dict['features']]
ndvi_values = [feat['properties']['NDVI'] for feat in rTable_dict['features']]
fitted_values = [feat['properties']['fitted'] for feat in rTable_dict['features']]

df = pd.DataFrame({'Date': dates, 'NDVI': ndvi_values, 'Fitted': fitted_values})

df = df.dropna()

if not df.empty:
    df['Date'] = pd.to_datetime(df['Date'])

    train_df = df[df['Date'] < '2023-05-01']
    valid_df = df[df['Date'] >= '2023-05-01']

    print(train_df)
    print(valid_df)

          Date      NDVI    Fitted
6   2022-01-16  0.453834  0.517946
18  2022-02-13  0.656015  0.547797
19  2022-02-15  0.565598  0.551148
26  2022-03-05  0.425886  0.587509
27  2022-03-07  0.541234  0.592121
28  2022-03-10  0.527651  0.599259
30  2022-03-15  0.510917  0.611570
31  2022-03-17  0.744119  0.616617
40  2022-04-09  0.831850  0.678319
45  2022-04-21  0.775561  0.710964
47  2022-04-26  0.832498  0.724218
50  2022-05-04  0.697603  0.744667
53  2022-05-11  0.836373  0.761483
55  2022-05-16  0.809823  0.772767
72  2022-06-28  0.787908  0.833558
73  2022-06-30  0.797773  0.834481
75  2022-07-05  0.806576  0.835987
76  2022-07-08  0.705711  0.836330
83  2022-07-25  0.750122  0.830344
84  2022-07-28  0.878596  0.827909
89  2022-08-09  0.841102  0.814295
90  2022-08-12  0.911660  0.809952
94  2022-08-22  0.646180  0.793048
116 2022-10-16  0.750000  0.657711
131 2022-11-22  0.622170  0.569615
141 2022-12-17  0.598784  0.533167
149 2023-01-06  0.586964  0.523260
154 2023-01-19  0.53